In [71]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
from time import sleep
import requests
import json
from pathlib import Path

In [72]:
#we want overlapping with> lvar, chip, atac. NOT other from gff, lnc

In [73]:
#still have to check to make sure no ranges overlap within type (eg atac)
#also compare with already selected list of enhancers

In [74]:
#we should also set a minimum length of a range to consider. 50 bp? lower range of enhancers

# important functions

In [75]:
def df2dic(al, intervals):
    ranges = { interval : defaultdict(list) for interval in intervals }
    for Var in intervals:
        temp_df = al[al["type"]==Var]
        for index, row in temp_df.iterrows():
            ranges[Var][row['chr']].append([row['start'],row['stop']])
    return ranges

In [76]:
def unzip(l):
    return list(zip(*l))

In [77]:
def range_merge(A, B):
    A0, A1 = A
    B0, B1 = B
    # return overlap, A', B'
    if( A1 < B0): # no overlap
        return [], [], B
    if(B1 < A0 ): # no overlap
        return [], A, []
    if ( A0 < B0 and B0 <= A1 and A1 <= B1) : # tail left
        return [B0, A1], [], [A1,B1]
    if (B0 <= A0 and A0 <= B1 and B1 < A1): # tail right
        return [A0, B1], [B1, A1], []
    if ( B0 <= A0 and A1 <= B1): # superset A ⊂ B
        return [A0, A1], [], [A1,B1]
    if (A0 < B0 and B1 < A1): # subset B ⊂ A
        return [B0, B1], [B1,A1], []
    else:
        print("ERROR:",A, B)

In [78]:
def interval_to_line(I,y):
    return list(range(I[0],I[1])), [y] * (I[1]-I[0])

def test_intervals(A,B):
    C, A_p, B_p = range_merge(A,B)
    plt.plot(*interval_to_line(A,1), label="A", color="red")
    plt.plot(*interval_to_line(B,0), label="B", color="blue")
    if len(C) > 0:
        plt.plot(*interval_to_line(C,-1), label="C", color="purple")
    if len(A_p) > 0:
        plt.plot(*interval_to_line(A_p,-2), label="A_p", color="red")
    if len(B_p) > 0:
        plt.plot(*interval_to_line(B_p,-3), label="B_p", color="blue")
    plt.legend()
    plt.show()

In [79]:
def compute_overlaps(As,Bs, debug=False):
    ai, bi = 0,0
    overlaps = []
    
    A = As[ai]
    B = Bs[bi]
    safety = 0
    max_safety = 1e6
    while True and safety < max_safety:
        safety += 1
        C, A_p, B_p = range_merge(A,B)

        if len(C) > 0:
            overlaps.append(C)
            if debug:
                print(A,B,C)

        if len(A_p) == 0:
            ai += 1
            if ai >= len(As):
                return overlaps
            else:
                A = As[ai]
        else:
            A = A_p

        if len(B_p) == 0:
            bi += 1
            if bi >= len(Bs):
                return overlaps
            else:
                B = Bs[bi]
        else:
            B = B_p
    
    if safety >= max_safety:
        print("YOU ARE A FAILURE")

In [80]:
def get_merge_results(all_chrs, al, kind1, kind2, ranges):
    ref=ranges[kind1]
    alt=ranges[kind2]
    results = pd.DataFrame() 
    for ch in all_chrs:
        if len(al[(al["chr"]==ch) & (al["type"]==kind1)]) > 0:
            new_contig = compute_overlaps(ref[ch], alt[ch], debug=False)
            for c in new_contig:
                results = results.append({'chr' : ch, 'start' : c[0], 'stop' : c[1]},  
                        ignore_index = True) 
    results.start = results.start.astype(int)
    results.stop = results.stop.astype(int)
    return results

In [81]:
def get_sub_results(all_chrs, al, kind1, kind2, ranges):
    ref=ranges[kind1]
    alt=ranges[kind2]
    results = pd.DataFrame() 
    for ch in all_chrs:
        if len(al[(al["chr"]==ch) & (al["type"]==kind1)]) > 0:
            new_contig = subtract_scaffold(ref[ch], alt[ch])
            for c in new_contig:
                results = results.append({'chr' : ch, 'start' : c[0], 'stop' : c[1]},  
                        ignore_index = True) 
    results.start = results.start.astype(int)
    results.stop = results.stop.astype(int)
    return results

In [82]:
def subtract_scaffold(contig, enemy, debug=False, max_safety=1e6):
    good_idx = 0
    evil_idx = 0
    safety = 0 #this is to avoid an infinite while loop

    while True and safety < max_safety:
        if(good_idx >= len(contig) or evil_idx >= len(enemy)):
            print("YOU ARE A FAILURE.")
            break
        good = contig[good_idx]
        evil = enemy[evil_idx]
        good_start, good_end = good[0], good[1]
        evil_start, evil_end = evil[0], evil[1]
        if debug:
            print(f"checking {good} {evil} :",end=" ")
        if(check_overlap(good,evil)):
            print("OVERLAPPED")
            print(good, evil)
            if debug:
                print("overlap")
            segments = range_takeout(good,evil)
            if debug:
                print("segments",segments)
            contig = contig[:good_idx] + segments + contig[good_idx+1:]
            if debug:
                print("new",contig[good_idx-1:good_idx+2], "current idx", contig[good_idx])
                
            if(good_idx >= len(contig) or evil_idx >= len(enemy)):
                print("extra check saved us.")
                break

        else:
            if(evil_end <= good_start):
                if debug:
                    print("evil")
                evil_idx += 1
                if(evil_idx >= len(enemy)):
                    if debug:
                        print("EVIL FINISHED")
                    break
                if debug:
                    print("next evil", enemy[evil_idx])
            else:
                if debug:
                    print("good")
                good_idx += 1
                if(good_idx >= len(contig)):
                    if debug:
                        print("GOOD FINISHED")
                    break
                if debug:
                    print("next good", contig[good_idx])

        safety += 1
    if debug:
        print(safety)
    return contig

In [83]:
#quick check if two regions overlap
def check_overlap(a, b):
    return max(0, min(a[1], b[1]) - max(a[0], b[0])) > 0

In [84]:
def range_takeout(contig, evil):
    A0, A1 = contig #region that we like, e.g. conserved region, start and end respectively, start being the lower number no matter the direction of the gene for example
    B0, B1 = evil #region that we don't want in the end, e.g. coding region, start and end
    if( A1 < B0 or B1 < A0 ): # no overlap
        return [contig]
    elif ( A0 < B0 and B0 <= A1 and A1 <= B1) : # tail left
        return [[A0, B0]]
    elif (B0 <= A0 and A0 <= B1 and B1 < A1): # tail right
        return [[B1, A1]]
    elif ( B0 <= A0 and A1 <= B1): # superset
        return []
    elif (A0 < B0 and B1 < A1): # subset
        return [[A0,B0],[B1,A1]]
    else:
        print("ERROR:",contig, evil)

# step 1: overlap between atac and chip

In [85]:
reference = "atac"
toadd = ["chip"]
intervals = [reference] + toadd
intervals

['atac', 'chip']

In [86]:
atac = pd.read_csv("atac_mapped_translatedto5.0.csv", header=None, names=["chr","start","stop"])
chip = pd.read_csv('AlxChip_mapped_translatedto5.0.txt', sep="\t", header=None, names=["chr","start","stop"])

#processing dataframes
atac["type"] = "atac"
chip["type"] = "chip"
atac=atac.sort_values(by=["chr",'start'])
chip=chip.sort_values(by=["chr",'start'])
al = pd.concat([atac, chip])
all_chrs=al.chr.unique()
al["length"] = al["stop"] - al["start"]

#CHECK IF THERE ARE NEGATIVE NUMBERS
min(al["length"]) <= 0

False

In [17]:
#make it into a dictionary 
ranges = df2dic(al, intervals)

In [18]:
results=get_merge_results(all_chrs, al, "atac", "chip", ranges)

In [19]:
len(results)

85

In [20]:
results

,chr,start,stop
0,NW_022145514.1,743495,743825
1,NW_022145516.1,1320975,1321168
2,NW_022145520.1,521014,521247
3,NW_022145527.1,183872,184137
4,NW_022145533.1,26015,26218
...,...,...,...
80,NW_022145615.1,10561119,10561413
81,NW_022145615.1,10634069,10634473
82,NW_022145615.1,10654052,10654428
83,NW_022145615.1,10671067,10671460


# step 2: overlap with lvar

In [21]:
reference = "atac_chip"
toadd = ["lvar"]
intervals = [reference] + toadd
intervals

['atac_chip', 'lvar']

In [22]:
lvar = pd.read_csv('lvar_spur_48_50_mapped_translatedto50.csv', header=None, names=["chr","start","stop"])

#processing dataframes
results["type"] = "atac_chip"
lvar["type"] = "lvar"
results=results.sort_values(by=["chr",'start'])
lvar=lvar.sort_values(by=["chr",'start'])
al = pd.concat([results, lvar])
all_chrs=al.chr.unique()
al["length"] = al["stop"] - al["start"]

#CHECK IF THERE ARE NEGATIVE NUMBERS
min(al["length"]) <= 0

False

In [23]:
#make it into a dictionary 
ranges = df2dic(al, intervals)

In [24]:
results=get_merge_results(all_chrs, al, "atac_chip", "lvar", ranges)

In [25]:
len(results)

20

In [26]:
results

,chr,start,stop
0,NW_022145514.1,743559,743624
1,NW_022145533.1,26157,26207
2,NW_022145594.1,40376691,40376779
3,NW_022145594.1,40376783,40376838
4,NW_022145594.1,40986392,40986480
5,NW_022145594.1,40986484,40986539
6,NW_022145594.1,41435943,41436041
7,NW_022145594.1,41436041,41436041
8,NW_022145594.1,41436098,41436159
9,NW_022145594.1,41436159,41436159


# step 3: take out lncRNA

In [27]:
reference = "atac_chip"
subtractables = ["lnc"]
intervals = [reference] + subtractables
intervals

['atac_chip', 'lnc']

In [28]:
lnc = pd.read_csv("lncRNA_mapped_translatedto5.0.csv", header=1, names=["chr","start","stop"])

#processing dataframes
results["type"] = "atac_chip"
lnc["type"] = "lnc"
results=results.sort_values(by=["chr",'start'])
lnc=lnc.sort_values(by=["chr",'start'])
al = pd.concat([results, lnc])
all_chrs=al.chr.unique()
al["length"] = al["stop"] - al["start"]

#CHECK IF THERE ARE NEGATIVE NUMBERS
min(al["length"]) < 0

False

In [29]:
#make it into a dictionary 
ranges = df2dic(al, intervals)

In [30]:
results = get_sub_results(all_chrs, al, "atac_chip", "lnc", ranges)

OVERLAPPED
[7246561, 7246617] [7237563, 7248308]
extra check saved us.


In [31]:
len(results)

19

In [32]:
results

,chr,start,stop
0,NW_022145514.1,743559,743624
1,NW_022145533.1,26157,26207
2,NW_022145594.1,40376691,40376779
3,NW_022145594.1,40376783,40376838
4,NW_022145594.1,40986392,40986480
5,NW_022145594.1,40986484,40986539
6,NW_022145594.1,41435943,41436041
7,NW_022145594.1,41436041,41436041
8,NW_022145594.1,41436098,41436159
9,NW_022145594.1,41436159,41436159


# step 4: take out gff

In [33]:
reference = "atac_chip"
subtractables = ["gff"]
intervals = [reference] + subtractables
intervals

['atac_chip', 'gff']

In [34]:
gff = pd.read_csv("nonoverlapping_nointronpromoter_gffannotations.csv", header=1, names=["chr","start","stop"])

#processing dataframes
results["type"] = "atac_chip"
gff["type"] = "gff"
results=results.sort_values(by=["chr",'start'])
gff=gff.sort_values(by=["chr",'start'])
al = pd.concat([results, gff])
all_chrs=al.chr.unique()
al["length"] = al["stop"] - al["start"]

#CHECK IF THERE ARE NEGATIVE NUMBERS
min(al["length"]) < 0

False

In [35]:
#make it into a dictionary 
ranges = df2dic(al, intervals)

In [36]:
results = get_sub_results(all_chrs, al, "atac_chip", "gff", ranges)

OVERLAPPED
[743559, 743624] [740646, 766916]
extra check saved us.
OVERLAPPED
[26157, 26207] [25674, 29604]
extra check saved us.
OVERLAPPED
[41435943, 41436041] [41397222, 41450048]
OVERLAPPED
[41436098, 41436159] [41397222, 41450048]
OVERLAPPED
[1831811, 1831892] [1815658, 1844774]
extra check saved us.
OVERLAPPED
[17786688, 17786773] [17785775, 17858592]
OVERLAPPED
[7609848, 7609905] [7604328, 7610750]
OVERLAPPED
[6959103, 6959165] [6958164, 6977917]
extra check saved us.
OVERLAPPED
[26098597, 26098647] [26095201, 26099130]
extra check saved us.
OVERLAPPED
[28567517, 28567573] [28559174, 28633632]
extra check saved us.
OVERLAPPED
[23771882, 23771947] [23767617, 23795245]
extra check saved us.


In [37]:
len(results)

8

In [38]:
results

,chr,start,stop
0,NW_022145594.1,40376691,40376779
1,NW_022145594.1,40376783,40376838
2,NW_022145594.1,40986392,40986480
3,NW_022145594.1,40986484,40986539
4,NW_022145594.1,41436041,41436041
5,NW_022145594.1,41436159,41436159
6,NW_022145599.1,26971910,26971971
7,NW_022145602.1,7609905,7609905


In [39]:
our_chrs = results.chr.unique()

# step 5: take out liftover from 3.1

In [40]:
def get_lift_data(scaffold):
    info=[]
    json_url = f"https://jbrowse.echinobase.org/JBrowse/data/sp5_0/tracks/sp3_1_lift/{scaffold}/trackData.json"
    resp = requests.get(url=json_url)
    if(resp.ok):
        data = resp.json()
        info.extend([{"chr":scaffold,"start":el[1],"end":el[2]} for el in data["intervals"]["nclist"]])
    else:
        print("error")
    
    return(info)

In [41]:
liftdf = pd.DataFrame()
for ch in our_chrs:
    temp=get_lift_data(ch)
    tdf = pd.DataFrame(temp)
    liftdf = pd.concat([liftdf,tdf])

In [42]:
liftdf

,chr,start,end
0,NW_022145594.1,11649,1300494
1,NW_022145594.1,1330313,2650609
2,NW_022145594.1,2651924,4039427
3,NW_022145594.1,4052024,4897638
4,NW_022145594.1,4886239,5049733
...,...,...,...
21,NW_022145602.1,26230698,27052751
22,NW_022145602.1,27077731,27540212
23,NW_022145602.1,27563853,28385855
24,NW_022145602.1,28402983,29237603


In [43]:
#making sure they are not overlapping

In [44]:
liftdf=liftdf.sort_values(by=["chr",'start'])
all_chrs=liftdf.chr.unique()

In [45]:
def clean_segments(src):
    if len(src) <= 1:
        return src
    
    a = src[0]
    dest = []
    for b in src[1:]:
        # print(a,b, check_overlap(a,b))
        if check_overlap(a,b):
            a = (min(a[0],b[0]),max(a[1],b[1]))
        else:
            dest.append(a)
            a = b
    dest.append(a)
    return dest

def check_overlap(a, b):
    return max(0, min(a[1], b[1]) - max(a[0], b[0])) > 0

def get_segs(df, chromosome):
    return [(start,stop) for _,(_,start,stop) in df[df['chr']==chromosome].iterrows()]

In [47]:
all_chrs

array(['NW_022145594.1', 'NW_022145599.1', 'NW_022145602.1'], dtype=object)

In [48]:
tresults = pd.DataFrame()
i=1
for ch in all_chrs:
    print(i)
    src = get_segs(liftdf,ch)
    tmp = clean_segments(src)
    for t in tmp:
        tresults = tresults.append({'chr' : ch, 'start' : t[0], 'stop' : t[1]}, ignore_index = True) 
    i=i+1

1
2
3


In [49]:
tresults

,chr,start,stop
0,NW_022145594.1,11649.0,1300494.0
1,NW_022145594.1,1330313.0,2650609.0
2,NW_022145594.1,2651924.0,4039427.0
3,NW_022145594.1,4052024.0,5049733.0
4,NW_022145594.1,5053585.0,5209929.0
...,...,...,...
116,NW_022145602.1,26230698.0,27052751.0
117,NW_022145602.1,27077731.0,27540212.0
118,NW_022145602.1,27563853.0,28385855.0
119,NW_022145602.1,28402983.0,29237603.0


In [50]:
tresults["start"]= tresults["start"].astype(int)
tresults["stop"]= tresults["stop"].astype(int)

In [51]:
tresults

,chr,start,stop
0,NW_022145594.1,11649,1300494
1,NW_022145594.1,1330313,2650609
2,NW_022145594.1,2651924,4039427
3,NW_022145594.1,4052024,5049733
4,NW_022145594.1,5053585,5209929
...,...,...,...
116,NW_022145602.1,26230698,27052751
117,NW_022145602.1,27077731,27540212
118,NW_022145602.1,27563853,28385855
119,NW_022145602.1,28402983,29237603


In [52]:
results

,chr,start,stop
0,NW_022145594.1,40376691,40376779
1,NW_022145594.1,40376783,40376838
2,NW_022145594.1,40986392,40986480
3,NW_022145594.1,40986484,40986539
4,NW_022145594.1,41436041,41436041
5,NW_022145594.1,41436159,41436159
6,NW_022145599.1,26971910,26971971
7,NW_022145602.1,7609905,7609905


In [53]:
reference = "atac_chip"
subtractables = ["lift"]
intervals = [reference] + subtractables
intervals

['atac_chip', 'lift']

In [54]:
#processing dataframes
results["type"] = "atac_chip"
tresults["type"] = "lift"
results=results.sort_values(by=["chr",'start'])
tresults=tresults.sort_values(by=["chr",'start'])
al = pd.concat([results, tresults])
all_chrs=al.chr.unique()
al["length"] = al["stop"] - al["start"]

#CHECK IF THERE ARE NEGATIVE NUMBERS
min(al["length"]) < 0

False

In [55]:
al

,chr,start,stop,type,length
0,NW_022145594.1,40376691,40376779,atac_chip,88
1,NW_022145594.1,40376783,40376838,atac_chip,55
2,NW_022145594.1,40986392,40986480,atac_chip,88
3,NW_022145594.1,40986484,40986539,atac_chip,55
4,NW_022145594.1,41436041,41436041,atac_chip,0
...,...,...,...,...,...
116,NW_022145602.1,26230698,27052751,lift,822053
117,NW_022145602.1,27077731,27540212,lift,462481
118,NW_022145602.1,27563853,28385855,lift,822002
119,NW_022145602.1,28402983,29237603,lift,834620


In [56]:
#make it into a dictionary 
ranges = df2dic(al, intervals)

In [57]:
results = get_sub_results(all_chrs, al, "atac_chip", "lift", ranges)

OVERLAPPED
[40376691, 40376779] [39157374, 41817495]
OVERLAPPED
[40376783, 40376838] [39157374, 41817495]
OVERLAPPED
[40986392, 40986480] [39157374, 41817495]
OVERLAPPED
[40986484, 40986539] [39157374, 41817495]
OVERLAPPED
[26971910, 26971971] [26624983, 27204512]
extra check saved us.


In [58]:
results

,chr,start,stop
0,NW_022145594.1,41436041,41436041
1,NW_022145594.1,41436159,41436159
2,NW_022145602.1,7609905,7609905


In [67]:
our_chrs = results.chr.unique()

In [68]:
our_chrs

array(['NW_022145594.1', 'NW_022145602.1'], dtype=object)

# step 6: overlap with TFBS

In [61]:
def get_TFBS_data(scaffold):
    info=[]
    i=1
    while True:
        json_url = f"https://jbrowse.echinobase.org/JBrowse/data/sp5_0/tracks/TFBS/{scaffold}/lf-{i}.json"
        resp = requests.get(url=json_url)
        if(resp.ok):
            data = resp.json()
            info.extend([{"chr":scaffold,"start":el[1],"end":el[2]} for el in data])
        else:
            print("done")
            print(i)
            return(info, i)
        sleep(0.2)
        i=i+1

In [70]:
tfdf = pd.DataFrame()
for ch in our_chrs:
    print(ch)
    temp=get_TFBS_data(ch)
    tdf = pd.DataFrame(temp)
    filename = f"TFBS_scaffold{ch}.csv"
    if(not Path(filename).exists()):
        tdf.to_csv(filename,index=False)
    tfdf = pd.concat([tfdf,tdf])

NW_022145594.1


KeyboardInterrupt: 